In [1]:
import os
import sys

# Init project path
PROJECT_DIR = os.getcwd() + "/../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2

In [14]:
from models import LightGBM

if len(sys.argv) == 2:
    config_file_name = sys.argv[1]
else:
    config_file_name = "lightgbm_0"

print("Config file Name: ", config_file_name)

# Parse args
config: dict = json.load(open("./../configs/{}.json".format(config_file_name)))
    
models: dict = {"lightgbm": LightGBM}
model = models[config["model"]["name"]]()

Config file Name:  lightgbm_0


In [15]:
import json
import sys

import numpy as np
import pandas as pd

from db import table_load
from models import LightGBM, Model
from utils import timer

cv_result = pd.DataFrame()
cv_models = []
n_splits = config["cv"]["n_splits"]
random_state = config["cv"]["random_state"]
target_cols = config["features"]["target"]
train_cols = config["features"]["train"]
categorical_cols = config["features"]["categorical"]
params = config["model"]

for n_fold in range(n_splits):
    with timer("CV No.{}".format(n_fold)):
        train = table_load(
            "cv_train_{}".format(n_fold), train_cols + target_cols
        )
        valid = table_load(
            "cv_test_{}".format(n_fold), train_cols + target_cols
        )

        cv_model, y_pred = model.train_and_predict(
            train=train,
            valid=valid,
            weight=None,
            categorical_features=categorical_cols,
            target_cols=target_cols,
            train_cols=train_cols,
            params=params,
        )

        cv_models.append(cv_model)

        y_real = valid[target_cols].iloc[:, 0].values.flatten()
        cv_result = pd.concat(
            [
                cv_result,
                pd.DataFrame(
                    {
                        "index": valid.index,
                        "predicted": y_pred.flatten(),
                        "real": y_real,
                        "difference": y_pred.flatten() - y_real,
                        "n_fold": n_fold,
                    }
                ),
            ]
        )

        predicted = (y_pred.flatten() > 0.5).astype(int)
        accuracy = (predicted == y_real).sum() / len(predicted)

        print("Accuracy: {}".format(accuracy))

[CV No.0] start
Accuracy: 0.7430167597765364
[CV No.0] done in 0.2459 s

[CV No.1] start
Accuracy: 0.8044692737430168
[CV No.1] done in 0.2102 s

[CV No.2] start
Accuracy: 0.7921348314606742
[CV No.2] done in 0.1973 s

[CV No.3] start
Accuracy: 0.8258426966292135
[CV No.3] done in 0.2462 s

[CV No.4] start
Accuracy: 0.7966101694915254
[CV No.4] done in 0.2058 s



In [8]:
y_pred

array([0.12096708, 0.75858035, 0.09775892, 0.0961602 , 0.76331944,
       0.41596917, 0.76944465, 0.6002301 , 0.31151882, 0.0850429 ,
       0.84498453, 0.64716528, 0.62601293, 0.83543819, 0.31151882,
       0.12096708, 0.75990298, 0.05222241, 0.29038332, 0.12635554,
       0.14239152, 0.72188893, 0.05222241, 0.13315208, 0.6043488 ,
       0.21107004, 0.09836593, 0.2712578 , 0.83687427, 0.31151882,
       0.69772148, 0.05222241, 0.07105649, 0.15217173, 0.54161161,
       0.19744159, 0.24020255, 0.70614422, 0.31151882, 0.24020255,
       0.65765017, 0.11731821, 0.67972448, 0.90872927, 0.08178991,
       0.24020255, 0.16447765, 0.0961602 , 0.0850429 , 0.0850429 ,
       0.95868403, 0.05426592, 0.31303275, 0.64761765, 0.62601293,
       0.25785554, 0.18117987, 0.71490184, 0.14105014, 0.69232926,
       0.62601293, 0.64380698, 0.5839984 , 0.10485393, 0.07830178,
       0.7054487 , 0.84575436, 0.10024103, 0.78625816, 0.84575436,
       0.78169037, 0.93922025, 0.90872927, 0.12096708, 0.63132